In [5]:
import requests
import time
import re
import psycopg2
import os

import plotly.plotly as py
import plotly.graph_objs as go

import cufflinks as cf
import pandas as pd

from scipy import stats

PLOTLY_UN    = os.environ.get("PLOTLY_UN")
PLOTLY_TOKEN = os.environ.get("PLOTLY_TOKEN")

POSTGRES_DB_NAME = os.environ.get("POSTGRES_DB_NAME")
POSTGRES_DB_UN   = os.environ.get("POSTGRES_DB_UN")
POSTGRES_DB_PW   = os.environ.get("POSTGRES_DB_PW")
POSTGRES_DB_HOST = os.environ.get("POSTGRES_DB_HOST")
connect_to_db = 'postgresql+psycopg2://' + \
                POSTGRES_DB_UN + ':' + POSTGRES_DB_PW + '@' + \
                POSTGRES_DB_HOST + '/' + POSTGRES_DB_NAME;

%load_ext sql
%config echo=False
%sql $connect_to_db
connection = psycopg2.connect(dbname=POSTGRES_DB_NAME, user=POSTGRES_DB_UN, password=POSTGRES_DB_PW);
cursor     = connection.cursor()


In [6]:
    %sql SELECT * FROM view_commit_data_search_for_CVE WHERE message ~ 'CVE-2014-1266'

2 rows affected.


[('a72e7f82540f7b45fb177759032336e5bcaf0dee', 'bassosimone', datetime.datetime(2014, 2, 22, 22, 44, 9, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=60, name=None)), 'bassosimone@gmail.com', 'Simone Basso', "Makefile.in: add -Wunreachable-code\n\nI was reading about the CVE-2014-1266 SSL/TLS Apple bug on\nImperialViolet and learnt that clang has a separat ... (64 characters truncated) ... cluded\nin the -Wall warnings [1].\n\nSo, let's add -Wunreachable-code to Makefile.in.\n\n[1] https://www.imperialviolet.org/2014/02/22/applebug.html", 2, 1),
 ('ab9b061b32c4897a36eddb17ca309dd4f6d8e68f', '', datetime.datetime(2016, 9, 12, 22, 13, 54, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=120, name=None)), 'maya', 'maya', 'add packages gcc6, gcc6-libs (GCC 6.2.0), based on wip/gcc6snapshot\nXXX add logic to mk/compiler/gcc.mk\n\nChanges from GCC5 series:\n\nCaveats\n\n  ... (35446 characters truncated) ...  [6 Regression] ICE in schedule_generic_params_dies_gen\n71056 [6 Regression] __builtin\n70540 [4.9/5 Regression] ICE on invalid code in cxx_incomple', 1415, 1)]

In [85]:
import plotly.plotly as ply
import plotly.graph_objs as go
import pandas as pd

query = """
SELECT cve, COUNT(a.sha) AS shacnt FROM (
    SELECT  sha, regexp_matches(message, 'CVE-[0-9]{3,4}-[0-9]{3,7}', 'g') as cve
    FROM view_commit_data_search_for_CVE) a
GROUP BY a.cve ORDER BY shacnt DESC LIMIT 30"""

df = pd.read_sql_query(query, connection)
ply.iplot({
    'data': [go.Bar(x=df.cve, y=df.shacnt)],
    'layout': {
        'title': "Mentions per CVE"
    }
}, filename="mentions-per-cve")

In [48]:
%%sql
SELECT cve, COUNT(a.sha) AS shacnt FROM (
    SELECT  sha, regexp_matches(message, 'CVE-[0-9]{3,4}-[0-9]{3,7}', 'g') as cve
    FROM view_commit_data_search_for_CVE) a
GROUP BY a.cve ORDER BY shacnt DESC
LIMIT 10

10 rows affected.


cve,shacnt
['CVE-2015-1805'],1696
['CVE-2014-3153'],1064
['CVE-2013-4312'],1044
['CVE-2017-7184'],1038
['CVE-2014-7970'],901
['CVE-2017-2618'],878
['CVE-2015-7550'],863
['CVE-2016-7097'],838
['CVE-2016-0774'],792
['CVE-2015-8543'],748


## CVE-2015-1805
The (1) pipe_read and (2) pipe_write implementations in fs/pipe.c in the Linux kernel before 3.16 do not properly consider the side effects of failed `__copy_to_user_inatomic` and `__copy_from_user_inatomic` calls, which allows local users to cause a denial of service (system crash) or possibly gain privileges via a crafted application, aka an "I/O vector array overrun." 

## CVE-2014-3153
The futex_requeue function in kernel/futex.c in the Linux kernel through 3.14.5 does not ensure that calls have two different futex addresses, which allows local users to gain privileges via a crafted FUTEX_REQUEUE command that facilitates unsafe waiter modification. 

## CVE-2013-4312
The Linux kernel before 4.4.1 allows local users to bypass file-descriptor limits and cause a denial of service (memory consumption) by sending each descriptor over a UNIX socket before closing it, related to net/unix/af_unix.c and net/unix/garbage.c. 

In [91]:
import plotly.plotly as ply
import plotly.graph_objs as go
import pandas as pd

query = "SELECT DATE(timestamp), COUNT(sha) AS shacount FROM view_commit_data_search_for_cve GROUP BY date ORDER BY date"
df = pd.read_sql_query(query, connection)
ply.iplot({
    'data': [go.Scatter(x=df.date, y=df.shacount)],
    'layout': {
        'title': "CVE reactions over time",
        'xaxis': dict(range = ['2012-01-01','2017-12-31'])
    }}, filename="cve-reactions-over-time")

In [126]:
import plotly.plotly as ply
import plotly.graph_objs as go
import pandas as pd
import datetime as dt

query = """
SELECT a.date, a.shacount/b.shacount::float AS shacount
FROM (
    SELECT DATE(timestamp) as date, COUNT(sha) AS shacount
    FROM view_commit_data_search_for_cve
    GROUP BY date
    ORDER BY date
) a
INNER JOIN
view_commit_data_commits_per_day b
ON a.date = b.date
"""

df = pd.read_sql_query(query, connection)

#df.date = pd.to_datetime(df.date)
#df.date = df.date.map(dt.datetime.toordinal)
#slope, intercept, r_value, p_value, std_err = stats.linregress(df.date,df.shacount)
#line = slope*df.date+intercept
#df.date = df.date.map(dt.datetime.toordinal)
#df.date = pd.to_datetime(df.date)

ply.iplot({
    'data': [
        go.Scatter(x=df.date, y=df.shacount)
    ],
    'layout': {
        'title': "Adjusted reactions to CVE entries over time",
        'xaxis': { 'range': ['2012-01-01','2017-12-31']},
        'yaxis': { 'range': [0, 0.003], 'title': 'Percentage of total commits on a given day', 'tickformat': '.2%'}
    }}, filename="cve-reactions-adjusted-over-time")

In [92]:
%%sql
SELECT year || '/' || month AS MONTHYEAR, COUNT(sha) AS shacnt FROM (
    SELECT EXTRACT(MONTH FROM timestamp) AS month,
           EXTRACT(YEAR FROM timestamp) AS year,
           sha
    FROM view_commit_data_search_for_cve) a
GROUP BY year, month
ORDER BY shacnt DESC
LIMIT 5

5 rows affected.


monthyear,shacnt
2017/7,5571
2017/6,5529
2017/5,4787
2017/8,4163
2016/10,3907


In [127]:
query1 = """SELECT 'commits per CVE' AS label, AVG(shacount) AS value
FROM (
    SELECT  regexp_matches(message, 'CVE-[0-9]{3,4}-[0-9]{3,7}', 'g') as cve, COUNT(sha) AS shacount
    FROM view_commit_data_search_for_CVE
    GROUP BY cve) a
"""
query2 = """SELECT 'CVE references per commit' AS label, AVG(cvecount) AS value
FROM (
    SELECT sha, COUNT(match) AS cvecount
    FROM (
        SELECT sha, regexp_matches(message, 'CVE-[0-9]{3,4}-[0-9]{3,7}', 'g') AS match
        FROM view_commit_data_search_for_CVE
) b
GROUP BY sha) a
"""

df1 = pd.read_sql_query(query1, connection)
df2 = pd.read_sql_query(query2, connection)
df = pd.concat([df1,df2])

ply.iplot({
    'data': [go.Bar(x=df.label, y=df.value)],
    'layout': {
        'title': "Commits per CVE vs. CVE references per commit"
    }
}, filename="commits-per-cve-vs-cve-references-per-commit")

In [134]:
%%sql
SELECT regexp_matches(message, 'CVE-[0-9]{3,4}-[0-9]{3,7}', 'g') as cve, COUNT(sha) AS shacount
FROM view_commit_data_search_for_CVE
WHERE DATE(timestamp) = '2015-12-28'
GROUP BY cve
ORDER BY shacount DESC
LIMIT 10

10 rows affected.


cve,shacount
['CVE-2015-6611'],12
['CVE-2015-3873'],7
['CVE-2015-6608'],6
['CVE-2015-6610'],4
['CVE-2015-3876'],2
['CVE-2015-6598'],2
['CVE-2015-3869'],2
['CVE-2015-3868'],2
['CVE-2015-3879'],2
['CVE-2015-6605'],2


In [160]:
%%sql
SELECT regexp_matches(message, 'CVE-[0-9]{3,4}-[0-9]{3,7}', 'g') as cve, COUNT(sha) AS shacount
FROM view_commit_data_search_for_CVE
WHERE DATE(timestamp) = '2016-10-14'
GROUP BY cve
ORDER BY shacount DESC
LIMIT 20

20 rows affected.


cve,shacount
['CVE-2016-7412'],8
['CVE-2016-7417'],8
['CVE-2016-5340'],8
['CVE-2016-7416'],8
['CVE-2016-7414'],8
['CVE-2016-7413'],8
['CVE-2016-7418'],8
['CVE-2016-6304'],7
['CVE-2016-6303'],7
['CVE-2016-2182'],7


# Basic system applications are most discussed

In [6]:
query = """SELECT cve[1], COUNT(a.sha) AS shacnt FROM (
    SELECT  sha, regexp_matches(message, 'CVE-[0-9]{3,4}-[0-9]{3,7}', 'g') as cve 
    FROM view_commit_data_search_for_CVE) a 
GROUP BY a.cve ORDER BY shacnt DESC 
LIMIT 250"""
endangeredSoftware = []

cursor.execute(query)

for line in cursor:
    requestUrl = "http://127.0.0.1:5000/api/cve/{0}".format(line[0])
    cveJson = requests.get(requestUrl).json()
    
    if cveJson != None:
        endangeredSoftware = endangeredSoftware + cveJson['vulnerable_configuration']

#endangeredSoftware = [line['title'] for line in endangeredSoftware]
# extract products and remove duplicates here
onlyNameMatch = re.compile("^([A-Za-z ]+)")
cleanedSoftwareNames = []

for n in endangeredSoftware:
    m = onlyNameMatch.match(n)
    if m:
        cleanedSoftwareNames.append(m.groups(0)[0].strip())
    else:
        cleanedSoftwareNames.append(n)

cleanedSoftwareNames = set(cleanedSoftwareNames)
cleanedSoftwareNames[1:20]

TypeError: expected string or buffer

In [12]:
%%sql
SELECT domain[1], COUNT(domain) AS ct
FROM view_cve_referring_tweets_extracted_domains
GROUP BY domain
ORDER BY ct DESC
LIMIT 25

25 rows affected.


domain,ct
web.nvd.nist.gov,15156
goo.gl,3817
twitter.com,2787
github.com,2160
lnkd.in,1104
oss.sios.com,1030
protocol96.com,903
vuldb.com,711
access.redhat.com,603
marc.ttias.be,538
